In [1]:
import sqlite3
import time
import datetime

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [2]:
conn = sqlite3.connect('tutorial.db') # open/create new database

In [4]:
c = conn.cursor() # specify cursor for database

In [5]:
def tableCreate():
    c.execute("CREATE TABLE stuffToPlot(ID INT, unix REAL, datestamp TEXT, keyword TEXT, value REAL)")

In [7]:
tableCreate()

OperationalError: table stuffToPlot already exists

In [10]:
def dataEntry():
    c.execute("INSERT INTO stuffToPlot VALUES(1, 1365876.98, '2016-01-09 12:54:21', 'Python Sentiment', 5)")
    c.execute("INSERT INTO stuffToPlot VALUES(2, 1365826.98, '2016-01-09 12:57:08', 'Python Sentiment', 5)")
    c.execute("INSERT INTO stuffToPlot VALUES(2, 136276.98, '2016-01-09 13:32:11', 'Python Sentiment', 4)")
    conn.commit()

In [11]:
dataEntry()

In [15]:
idfordb = 4
keyword = 'Python Sentiment'
value = 7

In [18]:
time.time() # unix time: no. of seconds after Jan 1 1970

1452957545.945306

In [19]:
datetime.datetime.fromtimestamp(time.time())

datetime.datetime(2016, 1, 16, 16, 20, 39, 134735)

In [23]:
datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

'2016-01-16 16:22:59'

In [ ]:
datetime.datetime.fromtimestamp(time.time()).strftime('%y-%m-%d')

In [28]:
def realDataEntry():
    date=datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
    c.execute("INSERT INTO stuffToPlot (ID, unix, datestamp, keyword, value) VALUES (?,?,?,?,?)",
             (idfordb, time.time(), date, keyword, value))
    conn.commit()

In [29]:
realDataEntry()

# Reading data

In [34]:
sql = "SELECT * FROM stuffToPlot WHERE keyword = ?"
# sql = "SELECT * FROM stuffToPlot WHERE keyword = ? AND source = ?"
# a potem
#     for row in c.execute(sql, [(wordUsed), (sourceVariable)]):



wordUsed ='Python Sentiment'

In [62]:
def readData():
    skipSymbols = str.maketrans('()','  ')
    for row in c.execute(sql, [(wordUsed)]):
        print(str(row).translate(skipSymbols))

In [58]:
readData()

 1, 1365876.98, '2016-01-09 12:54:21', 'Python Sentiment', 5.0 
 2, 1365826.98, '2016-01-09 12:57:08', 'Python Sentiment', 5.0 
 2, 136276.98, '2016-01-09 13:32:11', 'Python Sentiment', 4.0 
 4, 1452959471.316042, '2016-01-16 16:51:11', 'Python Sentiment', 7.0 


# Plot data

In [63]:
skipSymbols = str.maketrans('()','  ')

In [69]:
graphArray = []
for row in c.execute(sql, [(wordUsed)]):
    startingInfo = str(row).translate(skipSymbols)
    splitInfo = startingInfo.split(',')
    graphArrayAppend = splitInfo[2]+','+splitInfo[4]
    print(graphArrayAppend)
    graphArray.append(graphArrayAppend)

 '2016-01-09 12:54:21', 5.0 
 '2016-01-09 12:57:08', 5.0 
 '2016-01-09 13:32:11', 4.0 
 '2016-01-16 16:51:11', 7.0 


In [78]:
datestamp, value = np.loadtxt(graphArray, delimiter=',',
                              unpack=True, converters={ 0:mdates.strpdate2num(" %Y-%m-%d %H:%M:%S")}))

SyntaxError: invalid syntax (<ipython-input-78-57a766437e9c>, line 2)

In [80]:
c.execute("CREATE TABLE PDB_checklist(PDB_id TEXT, mod_date date, no_of_modRNA INT)")

OperationalError: table PDB_checklist already exists

In [85]:
c.execute("""SELECT * FROM information_schema.tables WHERE table_name = PDB_checklist""")

OperationalError: no such table: information_schema.tables

In [96]:
c.execute("SELECT * FROM sqlite_master")

In [94]:
c.fetchone()[0]

TypeError: 'NoneType' object is not subscriptable

In [130]:
a = c.execute("SELECT * FROM sqlite_master WHERE type='table' AND name='PDB_checklist'")

In [128]:
for row in a:
        print(str(row))

('table', 'PDB_checklist', 'PDB_checklist', 3, 'CREATE TABLE PDB_checklist(PDB_id TEXT, mod_date date, no_of_modRNA INT)')


In [131]:
print(a)

In [154]:
c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type='table' AND name='PDB_checklist'")

In [155]:
if c.fetchone()[0] == 1:
    print("There's one")
else:
    print("There's none")

There's one


In [156]:
c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type='table' AND name='PDB_checklsist'")

In [157]:
if c.fetchone()[0] == 1:
    print("There's one")
else:
    print("There's none")

There's none


In [4]:
def checkTableExist(cursor,tableName):
    """
    Checks if a table with a given name exists in the db, and returns
    a Boolean
    """
    cursor.execute("SELECT COUNT(*) FROM sqlite_master "
                   "WHERE type='table' "
                   "AND name=?", [tableName])
    if cursor.fetchone()[0] == 1:
        return True
    else:
        return False   

In [167]:
checkTableExist(c, 'PDB_checklist')

True

In [168]:
if not checkTableExist(c, "mod_RNA_entities"):
    c.execute("CREATE TABLE mod_RNA_entities(PDB_id TEXT, mod_date DATE, no_of_modRNA INT)")

In [219]:
c.execute("INSERT INTO PDB_checklist VALUES('1aa1', '2015-02-23', 2)")
c.execute("INSERT INTO PDB_checklist VALUES('2bb2', 'now', 3)")
conn.commit()

In [230]:
c.execute("SELECT COUNT(1) FROM PDB_checklist WHERE PDB_id = '22b2'")

In [231]:
c.fetchall()[0][0]

0

In [227]:
a[0][0]

1

In [257]:
c.execute("SELECT mod_date FROM PDB_checklist WHERE PDB_id = '1aa1'")

In [258]:
a = c.fetchall()

In [259]:
not a

False

In [262]:
a[0][0]

1990

# Update and delete
## 

In [277]:
def del_and_update():
    c.execute("SELECT * FROM stuffToPlot")
    [print(row) for row in c.fetchall()]
    
#     c.execute("UPDATE stuffToPlot SET value = 99 WHERE value =(?)", [99])
#     conn.commit()
    
#     c.execute("SELECT * FROM stuffToPlot")
#     [print(row) for row in c.fetchall()]   

#     c.execute("DELETE FROM stuffToPlot WHERE value = 99")
#     conn.commit()    
#     print(50*'#')

    c.execute("SELECT * FROM stuffToPlot WHERE value = 7")
    print(len(c.fetchall()))
    
    
    c.execute("SELECT * FROM stuffToPlot")
    [print(row) for row in c.fetchall()]

In [278]:
del_and_update()

(2, 136276.98, '2016-01-09 13:32:11', 'Python Sentiment', 4.0)
(4, 1452959471.316042, '2016-01-16 16:51:11', 'Python Sentiment', 7.0)
1
(2, 136276.98, '2016-01-09 13:32:11', 'Python Sentiment', 4.0)
(4, 1452959471.316042, '2016-01-16 16:51:11', 'Python Sentiment', 7.0)


# Check Real Modified RNA databases

In [2]:
connection = sqlite3.connect('modRNA.db')
c = connection.cursor()

In [11]:
if not checkTableExist(c, "mod_RNA_list"):
    c.execute("""CREATE TABLE
    mod_RNA_list(abbreviation TEXT, full_name TEXT)""")

In [85]:
modRNA_data = c.execute("""SELECT DISTINCT mono_abbreviation, mono_name
                           FROM mod_RNA_entities""")

In [86]:
# modRNA_data.fetchone()
modRNA = modRNA_data.fetchall()

In [87]:
for abbrev, name in modRNA:
    print(abbrev, name)

PSU PSEUDOURIDINE-5'-MONOPHOSPHATE
OMC O2'-METHYLYCYTIDINE-5'-MONOPHOSPHATE
OMG O2'-METHYLGUANOSINE-5'-MONOPHOSPHATE
1MG 1N-METHYLGUANOSINE-5'-MONOPHOSPHATE
5MC 5-METHYLCYTIDINE-5'-MONOPHOSPHATE


In [60]:
for (abbrev, name) in modRNA:
    a = c.execute("""SELECT DISTINCT PDB_id 
    FROM mod_RNA_entities WHERE mono_abbreviation = ?""", [abbrev])
    for b in a.fetchall():
        print(b[0])
    print()

1lu3

1luu
1lux

1luu
1lux

1luu

1luu
1lux



In [51]:
c.fetchall()

[('1luu',), ('1lux',)]

In [90]:
modRNA_data = c.execute("""SELECT abbreviation FROM mod_RNA_list""")

In [91]:
for abbrev in modRNA_data.fetchall():
    print(abbrev)

In [76]:
modRNA_data.fetchone()

In [81]:
for abbrev in modRNA:
    print(abbrev)

In [96]:
modRNA_data = c.execute("""SELECT DISTINCT mono_abbreviation
                           FROM mod_RNA_entities""")
for abbrev in modRNA_data.fetchall():
    print(abbrev)
    PDB_list = []
    c.execute("""SELECT DISTINCT PDB_id FROM mod_RNA_entities
                 WHERE mono_abbreviation = ?""", [(abbrev[0])])

('PSU',)
('OMC',)
('OMG',)
('1MG',)
('5MC',)


In [98]:
start_time = time.time()
for i in range(1000000):
    j = i
end_time = time.time()
elapsed_time = end_time - start_time
elapsed_time

0.07940506935119629